**ULMFiT model to Twitter US Airlines Sentiment**

In [ ]:
# importing libraries 
from fastai.text import * 

1. Data Wrangling

In [ ]:
# importing from csv 
dataset = pd.read_csv("Tweets.csv")
dataset.head()

2. Data Exploration

In [ ]:
# total number of tweets per each airline
tweetsCount = dataset.groupby(["airline"])["tweet_id"].count()
tweetsCount

In [ ]:
# horizontal bar graph showcasing the total number of tweets per each airline
tweetsCount.plot.barh()

In [ ]:
# total number of tweets per each sentiment
sentimentCount = dataset.groupby(["airline_sentiment"])["tweet_id"].count()
sentimentCount

In [ ]:
# vertical bar graph showcasing the total number of tweets per each airline
sentimentCount.plot.bar()

In [ ]:
# total number of tweets per each airline of each sentiment
tweetTypeCount = dataset.groupby(["airline", "airline_sentiment"])["tweet_id"].count()
tweetTypeCount

In [ ]:
# color coded bar graph showing the total number of tweets per each airline of each sentiment
tweetTypeCount.unstack().plot.bar()

3. Data Cleansing

In [ ]:
# subsetting tweet text and sentiment label for modeling
df = dataset[["airline_sentiment", "text"]]
df = df.rename(columns = {"airline_sentiment": "label"})
df.head()

In [ ]:
# helper functions to remove twitter handle name and unwanted characters from tweet text 
 
def removeHandleID(text):
  text = re.sub('@[^\s]+', '', text)
  return text

def removeExtraSpace(text):
  text = re.sub(' {2,}', '', text)
  return text

def removeHashTags(text):
  text = re.sub(r'([#])', r' \1 ', text)
  return text

In [ ]:
# cleaning the tweet texts 
df.text = df.text.apply(removeHandleID)
df.text = df.text.apply(removeExtraSpace)
df.text = df.text.apply(removeHashTags)
df.head()

In [ ]:
# splitting dataset into training and validation sets 
validDF = df.sample(frac = 0.2)
trainDF = df.drop(validDF.index)

4. Data Modeling

In [ ]:
# making data ready for modeling

path = ""
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = trainDF, 
                                  valid_df = validDF, 
                                  path = path)
# Classifier model data
data_clas = TextClasDataBunch.from_df(path = path, 
                                      train_df = trainDF, 
                                      valid_df = validDF, 
                                      vocab = data_lm.train_ds.vocab, 
                                      bs = 32)

In [ ]:
# saving the preprocessed vocabulary mappings
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [ ]:
# loading the language data model
data_lm = load_data(path, 'data_lm_export.pkl')
data_clas = load_data(path, 'data_clas_export.pkl', bs=16)

In [ ]:
# fine tuning language model 
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult = 0.5)
learn.fit_one_cycle(1, 1e-2)

In [ ]:
# unfreezing the model in order to fine-tune it
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

In [ ]:
# text completion prediction
tempText = learn.predict("missing baggage", n_words = 10)
tempText

In [ ]:
# saving model for transfer learning
learn.save_encoder('ft_enc')

In [ ]:
# loading ULMFit Model and building a classifier
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult = 0.5)
learn.load_encoder('ft_enc')

In [ ]:
# sample batch of classified data
data_clas.show_batch()

In [ ]:
# learning rate tuning
learn.fit_one_cycle(1, 1e-2)

In [ ]:
# unfreezing the model in order to fine-tune it
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

In [ ]:
# unfreezing the model in order to fine-tune it
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

5. Results

In [ ]:
# calculating accuracy from classification interpretation
interpret = TextClassificationInterpretation.from_learner(learn)
accuracy = accuracy(interpret.preds, interpret.y_true)
print("Accuracy: {0:.3f}%".format(accuracy*100))

In [ ]:
# plotting a confusion matirx
interpret.plot_confusion_matrix()
plt.title("Confusion Matrix")

In [ ]:
# sample test 
tempText
learn.predict(tempText)